In [1]:
import kotlin.random.Random

In [2]:
%useLatestDescriptors
%use lets-plot
%use dataframe

In [3]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.0.0.0-SNAPSHOT. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.4.0.

# Overriding aesthetics in guideLegend()¶
You can set specific aesthetic values for legend keys.

## Demo

In [4]:
val mpgDf = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/mpg.csv")
val mpgData = mpgDf.toMap()
mpgDf.head(3)

untitled,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,"1,800000",1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,"1,800000",1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,"2,000000",2008,4,manual(m6),f,20,31,p,compact


In [5]:
// By default alpha is not applied to the legend.

val baseMpgPlot = letsPlot(mpgData) { x = "displ"; y = "hwy"; color = "drv" } +
    geomPoint(size = 4, alpha = 0.2, stroke = 0)

baseMpgPlot

In [6]:
// Applying alpha and size for better look.

baseMpgPlot + guides(color = guideLegend(alpha = 0.4, size = 8.0))


## Experiments with parameters

In [7]:
Random(1)
val n = 4
val x = List(n) { it }
val y = List(n) { Random.nextDouble() }
val v = List(n) { Random.nextDouble() }
val data = mapOf("x" to x, "y" to y, "v" to v)

In [8]:
// Base plot
val p = letsPlot(data) { x = "x"; y = "y" } +
    geomPoint { color = asDiscrete("x") } +
    ggsize(400, 200)
p

In [9]:
// Specifying common aesthetics for all the legend labels.

p + guides(color = guideLegend(shape = 22, size = 8.0, stroke = 1.8, fill = "light-grey", alpha = 0.6))

In [10]:
// Using a list of values.

p + guides(color = guideLegend(color = listOf("red", "blue", "green", "pink")))

In [11]:
// Use `null` to keep the original value.

p + guides(color = guideLegend(color = listOf("red", null, "green", null)))

In [12]:
// Using a list that is smaller than the number of legend keys.
// The last value spreads to the rest of the keys.

p + guides(color = guideLegend(color = listOf("grey"), size = listOf(10.0, 8.0)))

In [13]:
// Using a list that is smaller than the number of legend keys.
// `null` can be used to stop value propagation; the rest keys will get their original values.

p + guides(color = guideLegend(color = listOf("grey", null), size = listOf(10.0, null, 8.0)))

## Reproduced an example from [rbind.io](https://aosmith.rbind.io/2020/07/09/ggplot2-override-aes/)


In [14]:
val dat = mapOf(
        "g1" to listOf("High", "Low", "High", "Low", "High", "Low", "High", "Low", "High", "Low", "High", "Low", "High", "Low", "High", "Low"),
        "g2" to listOf("Control", "Control", "Treatment", "Treatment", "Control", "Control", "Treatment", "Treatment", "Control", "Control", "Treatment", "Treatment", "Control", "Control", "Treatment", "Treatment"),
        "x" to listOf(0.42, 0.39, 0.56, 0.59, 0.17, 0.95, 0.85, 0.25, 0.31, 0.75, 0.58, 0.9, 0.6, 0.86, 0.61, 0.61),
        "y" to listOf(-1.4, 3.6, 1.1, -0.1, 0.5, 0.0, -1.8, 0.8, -1.1, -0.6, 0.2, 0.3, 1.1, 1.6, 0.9, -0.6)
    )

val bp = letsPlot(dat) { x = "x"; y = "y"; color = "g1"; shape = "g2" } +
        geomPoint(size = 5.0, alpha = 0.6)
bp

In [15]:
// Customized scales and default legend do not correspond well.

val cp = letsPlot(dat) { x = "x"; y = "y"; fill = "g1"; shape = "g2" } +
    geomPoint(size = 5.0, color = "black") +
    scaleFillManual(values = listOf("#002F70", "#EDB4B5")) +
    scaleShapeManual(values = listOf(21, 24))

cp

In [16]:
// This can be fixed by overriding aesthetics.

cp + guides(
    fill = guideLegend(shape = 21),
    shape = guideLegend(fill = "black")
)

## Testing

In [17]:
// `fill` and `size` are mapped to the same variable.
// The result is not defined if the same legend is set via different aesthetics.

val piePlot = letsPlot(mapOf("n" to listOf("a", "b", "c"))) + 
    geomPie(stat = Stat.identity) {fill = "n"; size = "n"}

gggrid(listOf(
    piePlot,
    piePlot + guides(
        fill = guideLegend(color = "blue"),
        size = guideLegend(stroke = 4.0, color = "black")
    )
)) + ggsize(800, 300)

In [18]:
// Verifying different aesthetics

p + guides(
    color = guideLegend(
        color = listOf("red", "blue", "green", "pink"),
        shape = 22,
        size = 8.0,
        stroke = 1.8,
        fill = "white",
        alpha = 0.4
    )
)

In [19]:
// Invalid or null values

p + guides(color=guideLegend(color = null, shape = -11, size = 0, stroke = -2))

In [20]:
// An empty list of values

p + guides(color=guideLegend(color=listOf<Int>()))

In [21]:
// `override_aes` in the case when some legend labels are combined into one;
// see issue https://github.com/JetBrains/lets-plot/issues/1129 for details.

val df = mapOf(
    "category" to listOf("A", "B", "C", "D"),
    "value" to listOf(10, 15, 20, 25)
)

letsPlot(df) + ggsize(400, 200) +
    geomPoint { x = "category"; y = "value"; color = "category" } +
    scaleColorManual(
        values = listOf("red", "blue", "green", "black"),
        labels = listOf("Label A", "Label B", "Label C", "Label C")
    ) +
    guides(color = guideLegend(color = listOf("pink", "gray", "green", "orange")))

## Custom legend

In [22]:
val data = mapOf(
    "x" to listOf(1, 2, 3, 4, 5),
    "y" to listOf(2, 3, 5, 7, 11)
)

// Verifying how `override_aes` works for custom legends.
// Note that the same effect can be achieved easier by using direct parameters.

val mPlot = letsPlot(data) { x = "x"; y = "y" } +
    geomPoint(manualKey = "Point") +
    geomLine(manualKey = "Line")

gggrid(
    listOf(
        mPlot,
        mPlot + guides(manual = guideLegend(size = 6.0)),
        mPlot + guides(manual = guideLegend(size = listOf(6.0, 2.0), color = listOf("red", "blue")))
    )
) + ggsize(900, 200)

In [23]:
// Changing the order in the legend via `index` in `layer_key`

val mPlot2 = letsPlot(data) { x = "x"; y = "y" } +
    geomPoint(manualKey = layerKey("Point", index = 1)) +
    geomLine(manualKey = layerKey("Line", index = 0))

gggrid(
    listOf(
        mPlot2,
        mPlot2 + guides(manual = guideLegend(size = listOf(2.0), color = listOf("red", "blue")))
    )
) + ggsize(600, 200)